# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
#
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

#
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

#
import gmpy2

#
import uuid

#
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 13282182480699984973456273021442731021441340402558181612070088145084033211159355636834885141571534322838214070600733942211292523583920491927507172646371101, g = 8490562976317989407365083208278195744008687630108694523551451653404212723647817246579679153464770349556278080348185359856504210968544330924989065193545090


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 56773725-3d52-43ed-8df9-aaf7988ca262 created
[TRACE] [DiffieHellmanUser] client a3b10e2e-13e4-426a-b209-94feea286170 created


In [10]:
# КЛюч получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [56773725-3d52-43ed-8df9-aaf7988ca262] x = mpz(2040580960055410159509033420379078139987239256578712570788792042049486063603810682644848344739492818645282041691003449705890238627718826269575993186411158)
[TRACE] [DiffieHellmanUser] [a3b10e2e-13e4-426a-b209-94feea286170] y = mpz(7645704810047422251096643115063602244244196468962977375522021299976957323716362522758705574953371859747038697288562187767077768200583244476184836954260501)
[TRACE] [DiffieHellmanUser] [a3b10e2e-13e4-426a-b209-94feea286170] derived key: af827b1425960c50d2e69a193fd132b1c63491262095fa7ce9c383273bba8b4d
[TRACE] [DiffieHellmanUser] [56773725-3d52-43ed-8df9-aaf7988ca262] derived key: af827b1425960c50d2e69a193fd132b1c63491262095fa7ce9c383273bba8b4d


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client c0ccaba0-af05-48dc-b2e7-1bcdd17782ae created


In [12]:
# Внедрившись в исполняющий протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [c0ccaba0-af05-48dc-b2e7-1bcdd17782ae] x = mpz(6668392746341720569700966112554868359902148728288159503214916053403315545571586080927771136062402057405704821160053388753305233832063614329030809578497095)
[TRACE] [DiffieHellmanUser] [a3b10e2e-13e4-426a-b209-94feea286170] y = mpz(3744427839303583213423607316714816250539177532251303425365863753298678527052547518882189636147876281130563915129159041112439540992664486378185716701734631)
[TRACE] [DiffieHellmanUser] [a3b10e2e-13e4-426a-b209-94feea286170] derived key: 9665ca79f95adc0f2b9c1660f1db2e35d27e03a4c3d7a46d1cc97ba4978a6499
[TRACE] [DiffieHellmanUser] [c0ccaba0-af05-48dc-b2e7-1bcdd17782ae] guessed x = mpz(1272579622011091594674737374173993053338880452392324638749210057339784586687266227472204081782785935664643621753869426389149515524482854633591949973901666)
[TRACE] [DiffieHellmanUser] [c0ccaba0-af05-48dc-b2e7-1bcdd17782ae] derived key: eb659f2e9d8f982e46e20cf2b228d4c0eff1823ec643770242cca9844c2a6293
